In [ ]:
## insall dependencies and refer to README

import openai

openai.api_key = 'OPENAI_API_KEY'

In [ ]:
## I set path in my workstation (you can possibly make the path from googld cloud_mount)

training_file_path = "Your_file_path"
validation_file_path = "Your_file_path"

training_file_name = "your_training_data_name.jsonl"
validation_file_name = "your_validation_data_name.jsonl"

In [ ]:
## following the file_path and set name above paragraph
## you can see training and validation ID after you've run this paragraph

training_response = openai.File.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
training_file_id = training_response["id"]

validation_response = openai.File.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune"
)
validation_file_id = validation_response["id"]

print("your_training_data_name file ID:", training_file_id)
print("Your_validation_data_name file ID:", validation_file_id)

In [ ]:
## You can check your status for uploading, processed and succeeded(train finished) after run this

file_status = openai.File.retrieve(training_file_id)
print(file_status['status'])

In [ ]:
## Your job ID and status will be shown after run this
## ** You have to wait for a minutes for uploading your files on Openai server and don't worry about not working (Patient for proceeding)
## You can set any names for suffix 

response = openai.FineTuningJob.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model="gpt-3.5-turbo",
    suffix="defective",
)

job_id = response["id"]

print("Job ID:", response["id"])
print("Status:", response["status"])

In [ ]:
## You can get Job ID, Status, and Trained Tokens after run this
## You don't have to worry about Tokens:None in that your training and validation datas already being train after run above paragraph

response = openai.FineTuningJob.retrieve(job_id)

print("Job ID:", response["id"])
print("Status:", response["status"])
print("Trained Tokens:", response["trained_tokens"])

In [ ]:
## This will shown the how training works through steps (If you want to check in real-time, you have to run several times)
## Step 160: training loss=0.00
## Step 170: training loss=0.00
## Step 180: training loss=1.19
## Example for your understanding

response = openai.FineTuningJob.list_events(id=job_id, limit=50)

events = response["data"]
events.reverse()

for event in events:
    print(event["message"])

In [ ]:
## You can get a fine-tuned model ID after finished above training steps
## I put over 200 columns for training and it took about less than half of minutes
## ID will be like that : Fine-tuned model ID: ft:gpt-3.5-turbo-0613:your_openai_ID_name: suffix_name: random_number

response = openai.FineTuningJob.retrieve(job_id)
fine_tuned_model_id = response["fine_tuned_model"]

print("Fine-tuned model ID:", fine_tuned_model_id)

In [ ]:
## This could be your own style. Openai used csv and pd for test but, i would make that simple.

import pandas as pd
from pprint import pprint

test_data = {
    'system_message': ["하자 전문가."],
    'user_message': ["벽지 하자 유형은 어떻게돼?"]
}

test_df = pd.DataFrame(test_data)

test_row = test_df.iloc[0]

system_message = test_row['system_message']

test_messages = []
test_messages.append({"role": "system", "content": system_message})

def create_user_message(row):
    return row['user_message']

user_message = create_user_message(test_row)
test_messages.append({"role": "user", "content": user_message})

pprint(test_messages)


In [ ]:
## You can use your fine-tuned model ID just like gpt-3.5-turbo 